# 该特征包括前两小时平均速度的数据、天气数据，日期数据
## 每个link单独学习

In [1]:
import numpy as np
import pandas as pd
import datetime

from sklearn import preprocessing

In [2]:
link_df = pd.read_csv('../../data/original/links (table 3).csv', index_col=0)

In [3]:
weather1_df = pd.read_csv('../../data/preprocess/preprocess_weather1.csv', index_col=0)
weather2_df = pd.read_csv('../../data/preprocess/preprocess_weather2.csv', index_col=0)
weather3_df = pd.read_csv('../../data/preprocess/preprocess_weather3.csv', index_col=0)
weather4_df = pd.read_csv('../../data/preprocess/preprocess_weather4.csv', index_col=0)

In [4]:
def weekdayOneHotEncoder(series):
    enc = preprocessing.OneHotEncoder()
    enc.fit([[0], [1], [2], [3], [4], [5], [6]])
    weekdays = []
    for t in series.values:
        weekdays.append([t])
    return enc.transform(weekdays).toarray()

In [5]:
def directionOneHotEncoder(series):
    enc = preprocessing.OneHotEncoder()
    enc.fit([[0], [1], [2], [3], [4], [5], [6], [7], [8]])
    directions = []
    for t in series.values:
        directions.append([t])
    return enc.transform(directions).toarray()

# 训练集 2016-07-19~2016-09-05 
# 测试集 2016-09-06~2016-09-12

In [16]:
# train_start_date = pd.to_datetime('2016-7-19')
# train_end_date = pd.to_datetime('2016-9-05')
# test_start_date = pd.to_datetime('2016-9-06')
# test_end_date = pd.to_datetime('2016-9-12')
# export_path = '../../data/offline/validation_9_6/'

# train_start_date = pd.to_datetime('2016-7-19')
# train_end_date = pd.to_datetime('2016-9-19')
# test_start_date = pd.to_datetime('2016-9-20')
# test_end_date = pd.to_datetime('2016-9-26')
# export_path = '../../data/offline/validation_9_20/'

train_start_date = pd.to_datetime('2016-7-19')
train_end_date = pd.to_datetime('2016-10-10')
test_start_date = pd.to_datetime('2016-10-11')
test_end_date = pd.to_datetime('2016-10-17')
export_path = '../../data/offline/validation_10_11/'

In [17]:
all_day_count = (test_end_date - train_start_date).days
train_day_count = (train_end_date - train_start_date).days
test_day_count = (test_end_date - test_start_date).days

In [18]:
weekdays = []
ranges = pd.date_range(start=train_start_date, end=test_end_date)
for d in ranges:
    weekdays.append(d.weekday())


# Morning LINKS 6:00-10:00

In [19]:
for link_id in link_df.index:
    time_df = pd.read_csv('../../data/process/link_average_arrive_time/morning/'+ str(link_id) + '.csv', index_col=0, parse_dates=True)
    time_df = time_df.loc[train_start_date:test_end_date]
    
    time_df['std'] = time_df[['0', '1', '2', '3', '4', '5']].std(axis=1)
    time_df['mean'] = time_df[['0', '1', '2', '3', '4', '5']].mean(axis=1)
    
    temp_df = pd.merge(time_df, weather1_df, how='left', left_index=True, right_index=True)
    time_df = pd.merge(temp_df, weather2_df, how='left', left_index=True, right_index=True)
    time_df['weekday'] = weekdays
    
    train_time_df = time_df.loc[train_start_date:train_end_date]
    test_time_df = time_df.loc[test_start_date:test_end_date]
    
    X_time_df = time_df[['0', '1', '2', '3', '4', '5', 'std', 'mean',
                           'pressure_x', 'sea_pressure_x', 'wind_speed_x', 'temperature_x', 
                           'rel_humidity_x', 'precipitation_x','pressure_y', 'sea_pressure_y', 
                           'wind_speed_y', 'temperature_y', 'rel_humidity_y', 'precipitation_y',
                          ]]
    
    y_train_time_df = train_time_df[['6', '7', '8', '9', '10', '11']]
    y_test_time_df = test_time_df[['6', '7', '8', '9', '10', '11']]
    X_test_time_df = test_time_df[['0', '1', '2', '3', '4', '5']]

    tempa = preprocessing.scale(X_time_df)
    tempb = weekdayOneHotEncoder(time_df['weekday'])
    tempc = directionOneHotEncoder(time_df['wind_direction_x'])
    tempd = directionOneHotEncoder(time_df['wind_direction_y'])
    temp_df = pd.DataFrame(np.concatenate((tempa, tempb, tempc, tempd), axis=1))
    
    X_train_df = temp_df.loc[np.array(range(train_day_count + 1))]
    X_train_df.to_csv(export_path + 'feature/feature1/morning/X_train_link' + str(link_id) + '.csv')
    y_train_df = pd.DataFrame(y_train_time_df.values)
    y_train_df.to_csv(export_path + 'feature/feature1/morning/y_train_link' + str(link_id) + '.csv')
    
    X_test_df = temp_df.loc[np.array(range(train_day_count, train_day_count + test_day_count + 1))]
    y_test_df = pd.DataFrame(y_test_time_df.values)
    X_test_df.reset_index(drop=True).to_csv(export_path + 'feature/feature1/morning/X_test_link' + str(link_id) + '.csv')
    y_test_df.reset_index(drop=True).to_csv(export_path + 'feature/feature1/morning/y_test_link' + str(link_id) + '.csv')
    
    pd.DataFrame(X_test_time_df.values).reset_index(drop=True).to_csv(export_path + 'feature/feature1/morning/X_src_test_link' + str(link_id) + '.csv')
    

# Afternoon LINKS 15:00-19:00

In [20]:
for link_id in link_df.index:
    time_df = pd.read_csv('../../data/process/link_average_arrive_time/afternoon/'+ str(link_id) + '.csv', index_col=0, parse_dates=True)
    time_df = time_df.loc[train_start_date:test_end_date]
    
    time_df['std'] = time_df[['0', '1', '2', '3', '4', '5']].std(axis=1)
    time_df['mean'] = time_df[['0', '1', '2', '3', '4', '5']].mean(axis=1)
    
    temp_df = pd.merge(time_df, weather1_df, how='left', left_index=True, right_index=True)
    time_df = pd.merge(temp_df, weather2_df, how='left', left_index=True, right_index=True)
    time_df['weekday'] = weekdays
    
    train_time_df = time_df.loc[train_start_date:train_end_date]
    test_time_df = time_df.loc[test_start_date:test_end_date]
    
    X_time_df = time_df[['0', '1', '2', '3', '4', '5', 'std', 'mean',
                                                   'pressure_x', 'sea_pressure_x', 'wind_speed_x', 'temperature_x', 
                                                   'rel_humidity_x', 'precipitation_x','pressure_y', 'sea_pressure_y', 
                                                   'wind_speed_y', 'temperature_y', 'rel_humidity_y', 'precipitation_y']]
    
    y_train_time_df = train_time_df[['6', '7', '8', '9', '10', '11']]
    y_test_time_df = test_time_df[['6', '7', '8', '9', '10', '11']]
    X_test_time_df = test_time_df[['0', '1', '2', '3', '4', '5']]
    
    tempa = preprocessing.scale(X_time_df)
    tempb = weekdayOneHotEncoder(time_df['weekday'])
    tempc = directionOneHotEncoder(time_df['wind_direction_x'])
    tempd = directionOneHotEncoder(time_df['wind_direction_y'])
    temp_df = pd.DataFrame(np.concatenate((tempa, tempb, tempc, tempd), axis=1))
    
    X_train_df = temp_df.loc[np.array(range(train_day_count + 1))]
    X_train_df.to_csv(export_path + 'feature/feature1/afternoon/X_train_link' + str(link_id) + '.csv')
    y_train_df = pd.DataFrame(y_train_time_df.values)
    y_train_df.to_csv(export_path + 'feature/feature1/afternoon/y_train_link' + str(link_id) + '.csv')
    
    X_test_df = temp_df.loc[np.array(range(train_day_count, train_day_count + test_day_count + 1))]
    y_test_df = pd.DataFrame(y_test_time_df.values)
    X_test_df.reset_index(drop=True).to_csv(export_path + 'feature/feature1/afternoon/X_test_link' + str(link_id) + '.csv')
    y_test_df.reset_index(drop=True).to_csv(export_path + 'feature/feature1/afternoon/y_test_link' + str(link_id) + '.csv')
    
    pd.DataFrame(X_test_time_df.values).reset_index(drop=True).to_csv(export_path + 'feature/feature1/afternoon/X_src_test_link' + str(link_id) + '.csv')